Abbreviations of morphological classes used in folder structure and annotation file
----------------------------------------------------------------------------------------
**BAS** Basophil

**EBO** Erythroblast (and Monoblast)

**EOS** Eosinophil

**LYT** Lymphocyte (typical and atypical)

**MON** Monocyte

**MYO** Myeloblast (Metamyelocyte and Myelocyte)

**NGB** Neutrophil (band)

**NGS** Neutrophil (segmented)

**PMO** Promyelocyte (bilobled and not)

In [1]:
print('hello world')

hello world
